# Prepare data for finetuning the U-Net model

In [1]:
import sys
import os
import shutil
import glob
import random

import geopandas as gpd
from shapely.geometry import box
import rasterio
import rasterio.mask
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
wbt_path = "C:/Users/holgerv/WBT"
if wbt_path not in sys.path:
    sys.path.insert(1, wbt_path)

In [3]:
from whitebox_tools import WhiteboxTools

In [4]:
whitebox = WhiteboxTools()
whitebox.set_whitebox_dir(wbt_path)

In [5]:
whitebox.set_verbose_mode(False)

0

In [6]:
whitebox.set_compress_rasters(True)

0

In [7]:
basepath = "D:/users/holgerv/Ditches"

In [8]:
# Finetuning data directory
finetuning_dir = f"{basepath}/working/deep_learning/data/finetuning"

## Create model input grid cells

In [9]:
# Create grid cells from input GeoDataFrame bounds
def create_grid_from_bounds(in_gdf: gpd.GeoDataFrame, grid_size: int, interval: int) -> gpd.GeoDataFrame:
    
    # Total bounds of input GeoDataFrame
    xmin, ymin, xmax, ymax = in_gdf.total_bounds
    
    # Create list of grid cell polygons
    grid_cells = []
    for x in range(int(xmin), int(xmax), interval):
        for y in range(int(ymin), int(ymax), interval):
            if x + grid_size <= xmax:
                if y + grid_size <= ymax:
                    grid_cells.append(box(x, y, x + grid_size, y + grid_size))
                    
    # Create GeoDataFrame
    out_gdf = gpd.GeoDataFrame(grid_cells, columns=["geometry"], crs=in_gdf.crs)
    
    # Add ID column
    out_gdf["tile_id"] = out_gdf.index + 1
    
    return out_gdf

In [10]:
# Input DEM cell size
cell_size = 1

# Output grid size
grid_size = int(500 * cell_size)

# Grid cell interval
interval = grid_size

In [11]:
%%time

# Loop over map sheets
map_sheets = [53972, 54591]
for map_sheet in map_sheets:
    
    # Read existing tiles
    fp = f"{basepath}/working/deep_learning/data/estonia_digitization/{map_sheet}/{map_sheet}_grid_{grid_size}m_digitization.gpkg"
    in_gdf = gpd.read_file(fp)
    
    # Create new tiles
    out_gdf = create_grid_from_bounds(in_gdf, grid_size, interval)

    # Add map sheet number to tile ID
    out_gdf["tile_id"] = f"{map_sheet}_" + out_gdf["tile_id"].astype(str)
    
    # Write to GPKG
    out_gdf.to_file(f"{finetuning_dir}/{map_sheet}_grid_{grid_size}m_model.gpkg")

CPU times: total: 266 ms
Wall time: 2.72 s


## Split DEM to tiles

In [12]:
# Output directory
out_dirpath = f"{finetuning_dir}/dem"
if os.path.exists(out_dirpath):
    shutil.rmtree(out_dirpath)
os.mkdir(out_dirpath)

In [13]:
# Input DEM cell size
cell_size = 1

# Output grid size
grid_size = int(500 * cell_size)

In [14]:
%%time

# Loop over map sheets
map_sheets = [53972, 54591]
for map_sheet in map_sheets:
    
    # Read tiles
    fp = f"{finetuning_dir}/{map_sheet}_grid_{grid_size}m_model.gpkg"
    tiles = gpd.read_file(fp)
    
    # Loop over tiles
    for tile_id in tqdm(tiles["tile_id"], position=0, leave=True):
        
        # Read DEM raster based on tile mask
        shape = tiles.loc[tiles["tile_id"] == tile_id].geometry.values[0]
        with rasterio.open(f"Z:/Ditches/working/deep_learning/data/dem_1m_wbt/{map_sheet}_dem_1m_wbt.tif") as src:
            out_image, out_transform = rasterio.mask.mask(src, [shape], crop=True)
            
            # Output metadata
            out_meta = src.meta
            out_meta.update(
                {
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform
                }
            )
            
            # Write to GeoTIFF
            with rasterio.open(f"{out_dirpath}/{tile_id}.tif", "w", **out_meta) as dst:
                dst.write(out_image)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:10<00:00,  3.60it/s]

CPU times: total: 5.27 s
Wall time: 20 s


## Split vector ditches to tiles

In [15]:
# Output directory
out_dirpath = f"{finetuning_dir}/ditches_vector"
if os.path.exists(out_dirpath):
    shutil.rmtree(out_dirpath)
os.mkdir(out_dirpath)

In [16]:
%%time

# Loop over map sheets
map_sheets = [53972, 54591]
for map_sheet in map_sheets:
    
    # Read tiles
    fp = f"{finetuning_dir}/{map_sheet}_grid_{grid_size}m_model.gpkg"
    tiles = gpd.read_file(fp)
    
    # Read digitized ditches
    ditches = gpd.read_file(f"{basepath}/working/deep_learning/data/estonia_digitization/{map_sheet}/{map_sheet}_digitized_ditches.gpkg")
    
    # Add ID column
    ditches["id"] = ditches.index + 1
    
    # Intersect ditches with tiles
    ditches = gpd.overlay(ditches, tiles, how="intersection")
    
    # Explode multi-part geometries
    ditches = ditches.explode(index_parts=False).reset_index(drop=True)
    
    # Update ID column
    ditches["id"] = ditches.index + 1
    
    # Group by tile ID and write to GPKG
    for tile_id, group in tqdm(ditches.groupby("tile_id"), position=0, leave=True):
        group.to_file(f"{out_dirpath}/{tile_id}.shp", crs=ditches.crs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 94.63it/s]
<timed exec>:16: UserWarning: `keep_geom_type=True` in overlay resulted in 2 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 90.37it/s]

CPU times: total: 922 ms
Wall time: 953 ms


## Generate labels

Workflow for generating labels from digitized ditches:
1. Rasterize ditches based on the bounds of the corresponding DEM file
2. Create a binary raster of ditches, where the value 1 to indicates ditch and 0 non-ditch cells
3. Generate HPMF raster from DEM file
4. Reclassify HPMF values below -0.075 to 1
5. Buffer ditches by 3 m
6. Extract reclassified HPMF cells based on buffered ditches
7. Apply majority filter to buffers to remove spurious masked HPMF cells that are not connected to ditches
8. Flag cells that were digitized or are below the HPMF threshold as ditch cells

In [17]:
# Generate output directories
out_dirpaths= [
    f"{finetuning_dir}/ditches_raster",
    f"{finetuning_dir}/ditches_reclassified",
    f"{finetuning_dir}/hpmf",
    f"{finetuning_dir}/hpmf_reclassified",
    f"{finetuning_dir}/ditches_buffered",
    f"{finetuning_dir}/hpmf_masked",
    f"{finetuning_dir}/ditches_filtered",
    f"{finetuning_dir}/labels"
]
for out_dirpath in out_dirpaths:
    if os.path.exists(out_dirpath):
        shutil.rmtree(out_dirpath)
    os.mkdir(out_dirpath)

In [18]:
%%time

# Loop over vector ditches
ditches_vector_files = glob.glob(f"{finetuning_dir}/ditches_vector/*.shp")
for ditches_vector_file in tqdm(ditches_vector_files, position=0, leave=True):
    
    tile_id = os.path.basename(ditches_vector_file).split(".")[0]
    
    # Rasterize ditches
    field = "id"
    nodata = 0
    whitebox.vector_lines_to_raster(
        f"{finetuning_dir}/ditches_vector/{tile_id}.shp",
        f"{finetuning_dir}/ditches_raster/{tile_id}.tif",
        field=field,
        nodata=nodata,
        base=f"{finetuning_dir}/dem/{tile_id}.tif"
    )
    
    # Assign new value to ditch cells
    new_value = 1.0
    from_value = 1
    to_less_than = ditches["id"].max() + 1
    reclass_vals = f"{new_value};{from_value};{to_less_than}"
    whitebox.reclass(
        f"{finetuning_dir}/ditches_raster/{tile_id}.tif",
        f"{finetuning_dir}/ditches_reclassified/{tile_id}.tif",
        reclass_vals
    )

    # Generate HPMF raster
    filterx = 5
    filtery = 5
    whitebox.high_pass_median_filter(
        f"{finetuning_dir}/dem/{tile_id}.tif",
        f"{finetuning_dir}/hpmf/{tile_id}.tif",
        filterx=filterx,
        filtery=filtery,
        sig_digits=2
    )

    # Reclassify HPMF values below -0.075 to 1
    threshold = -0.075
    whitebox.less_than(
        f"{finetuning_dir}/hpmf/{tile_id}.tif",
        threshold,
        f"{finetuning_dir}/hpmf_reclassified/{tile_id}.tif"
    )

    # Buffer ditches by 3 m
    input = f"{finetuning_dir}/ditches_reclassified/{tile_id}.tif"
    buff_size_m = 3
    cell_size_m = rasterio.open(input).res[0]
    size = int(buff_size_m / cell_size_m)
    whitebox.buffer_raster(
        input,
        f"{finetuning_dir}/ditches_buffered/{tile_id}.tif",
        str(size)
    )

    # Extract reclassified HPMF cells based on buffered ditches
    whitebox.multiply(
        f"{finetuning_dir}/hpmf_reclassified/{tile_id}.tif",
        f"{finetuning_dir}/ditches_buffered/{tile_id}.tif",
        f"{finetuning_dir}/hpmf_masked/{tile_id}.tif"
    )

    # Apply majority filter
    filterx = 3
    filtery = 3
    whitebox.majority_filter(
        f"{finetuning_dir}/hpmf_masked/{tile_id}.tif",
        f"{finetuning_dir}/ditches_filtered/{tile_id}.tif",
        filterx=filterx,
        filtery=filtery
    )

    # Flag cells that were digitized or are below the HPMF threshold as ditch cells
    whitebox.Or(
        f"{finetuning_dir}/ditches_reclassified/{tile_id}.tif",
        f"{finetuning_dir}/ditches_filtered/{tile_id}.tif",
        f"{finetuning_dir}/labels/{tile_id}.tif"
    )

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:38<00:00,  1.86it/s]

CPU times: total: 1.36 s
Wall time: 38.8 s


## Normalize HPMF rasters

In [19]:
# Read HPMF statistics
stats_df = pd.read_csv(f"{basepath}/working/deep_learning/data/dem_1m_hpmf_stats.csv", encoding="utf-8")

In [20]:
stats_df

,file,hpmf_min,hpmf_max,hpmf_mean,hpmf_percentile_1,hpmf_percentile_10,hpmf_percentile_90,hpmf_percentile_99
0,D:/users/holgerv/Ditches/working/deep_learning...,-0.76,0.93,0.000417,-0.390000,-0.24,-0.09,-0.09
1,D:/users/holgerv/Ditches/working/deep_learning...,-1.90,1.08,0.000589,-0.440000,-0.25,-0.09,-0.09
2,D:/users/holgerv/Ditches/working/deep_learning...,-1.66,1.62,0.002162,-0.470000,-0.31,-0.10,-0.10
3,D:/users/holgerv/Ditches/working/deep_learning...,-1.50,1.44,0.004625,-0.410000,-0.25,-0.09,-0.09
4,D:/users/holgerv/Ditches/working/deep_learning...,-0.96,2.30,0.006886,-0.410000,-0.21,-0.09,-0.09
...,...,...,...,...,...,...,...,...
2068,D:/users/holgerv/Ditches/working/deep_learning...,-0.52,0.81,0.008336,-0.310000,-0.18,-0.09,-0.09
2069,D:/users/holgerv/Ditches/working/deep_learning...,-0.98,1.09,0.003026,-0.330000,-0.15,-0.08,-0.08
2070,D:/users/holgerv/Ditches/working/deep_learning...,-0.65,0.78,0.002513,-0.290000,-0.16,-0.08,-0.07
2071,D:/users/holgerv/Ditches/working/deep_learning...,-0.58,0.41,0.006919,-0.340000,-0.18,-0.09,-0.09


In [21]:
stats_df["hpmf_min"].describe(percentiles=[0.01])

count    2073.000000
mean       -1.237062
std         0.735341
min       -18.440000
1%         -4.028800
50%        -1.090000
max        -0.110000
Name: hpmf_min, dtype: float64

In [22]:
stats_df["hpmf_max"].describe(percentiles=[0.99])

count    2073.000000
mean        1.482142
std         1.000186
min         0.090000
50%         1.260000
99%         5.249200
max        14.690000
Name: hpmf_max, dtype: float64

In [23]:
# Get lower HPMF boundary based on first percentile of minimum values
lower_boundary = stats_df["hpmf_min"].describe(percentiles=[0.01])["1%"]

In [24]:
# Get upper HPMF boundary based on 99th percentile of maximum values
upper_boundary = stats_df["hpmf_max"].describe(percentiles=[0.99])["99%"]

In [25]:
display(lower_boundary)
display(upper_boundary)

-4.0288

5.249200000000028

In [26]:
# Normalize HPMF raster
def normalize_hpmf_raster(in_fp: str, lower_boundary: float, upper_boundary: float, out_fp: str):
    
    with rasterio.open(in_fp) as src:
        
        # Read array
        array = src.read(1)
        
        # Get profile
        out_profile = src.profile
        
        # Replace no data values
        out_nodata = np.nan
        array = np.where(array == out_profile["nodata"], np.nan, array)
        
        # Cap outliers below lower boundary
        array = np.where(array <= lower_boundary, lower_boundary, array)
        
        # Cap outliers above upper boundary
        array = np.where(array >= upper_boundary, upper_boundary, array)
        
        # Normalize based on lower and upper boundaries
        array_norm = (array - lower_boundary) / (upper_boundary - lower_boundary)
        
        # Update profile
        out_profile["nodata"] = out_nodata
        
        # Write to raster
        with rasterio.open(out_fp, "w", **out_profile) as dst:
            dst.write(array_norm, 1)
            
    return

In [27]:
# Output directory
out_dirpath = f"{finetuning_dir}/hpmf_norm"
if os.path.exists(out_dirpath):
    shutil.rmtree(out_dirpath)
os.mkdir(out_dirpath)

In [28]:
%%time

# Loop over HPMF files and normalize them
files = glob.glob(f"{finetuning_dir}/hpmf/*.tif")
for file in tqdm(files, position=0, leave=True):
    out_fp = f"{out_dirpath}/{os.path.basename(file)}"
    normalize_hpmf_raster(file, lower_boundary, upper_boundary, out_fp)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:01<00:00, 36.92it/s]

CPU times: total: 1.95 s
Wall time: 1.95 s


## Split data into training and test sets

In [29]:
# Input HPMF files
fp_hpmf_list = sorted(glob.glob(f"{finetuning_dir}/hpmf_norm/*.tif"))
hpmf_files = [os.path.basename(fp_hpmf) for fp_hpmf in fp_hpmf_list]

# Input labels
fp_labels_list = sorted(glob.glob(f"{finetuning_dir}/labels/*.tif"))
labels_files = [os.path.basename(fp_labels) for fp_labels in fp_labels_list]

# Loop over labels and collect matching image pairs
image_pairs = {}
for i in range(len(labels_files)):
    labels_file = labels_files[i]
    if labels_file in hpmf_files:
        fp_hpmf = glob.glob(f"{finetuning_dir}/hpmf_norm/{labels_file}")[0]
        fp_labels = glob.glob(f"{finetuning_dir}/labels/{labels_file}")[0]
        image_pairs[fp_hpmf] = fp_labels

In [30]:
pct_train = 0.8
pct_test = 1 - pct_train
n_samples = len(image_pairs)
n_train = round(n_samples * pct_train)
n_test = n_samples - n_train

In [31]:
n_train, n_test

(58, 14)

In [32]:
# Split HPMF files into training and test samples
random.seed(0)
fp_hpmf_list_train = random.sample(list(image_pairs.keys()), n_train)
fp_hpmf_list_test = [file for file in list(image_pairs.keys()) if file not in fp_hpmf_list_train]

In [33]:
len(fp_hpmf_list_train), len(fp_hpmf_list_test)

(58, 14)

### Generate training data directory

In [34]:
# Output directories
out_dir = f"{finetuning_dir}/training"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
out_dir_hpmf = f"{out_dir}/hpmf"
if os.path.exists(out_dir_hpmf):
    shutil.rmtree(out_dir_hpmf)
os.mkdir(out_dir_hpmf)
out_dir_labels = f"{out_dir}/labels"
if os.path.exists(out_dir_labels):
    shutil.rmtree(out_dir_labels)
os.mkdir(out_dir_labels)

In [35]:
%%time

# Copy image pairs to corresponding directories
for fp_hpmf in fp_hpmf_list_train:
    fp_labels = image_pairs[fp_hpmf]
    shutil.copy(fp_hpmf, out_dir_hpmf)
    shutil.copy(fp_labels, out_dir_labels)

CPU times: total: 78.1 ms
Wall time: 475 ms


In [36]:
len(os.listdir(out_dir_hpmf)) == n_train

True

In [37]:
len(os.listdir(out_dir_labels)) == n_train

True

### Generate test data directory

In [38]:
# Output directories
out_dir = f"{finetuning_dir}/testing"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
out_dir_hpmf = f"{out_dir}/hpmf"
if os.path.exists(out_dir_hpmf):
    shutil.rmtree(out_dir_hpmf)
os.mkdir(out_dir_hpmf)
out_dir_labels = f"{out_dir}/labels"
if os.path.exists(out_dir_labels):
    shutil.rmtree(out_dir_labels)
os.mkdir(out_dir_labels)

In [39]:
%%time

# Copy image pairs to corresponding directories
for fp_hpmf in fp_hpmf_list_test:
    fp_labels = image_pairs[fp_hpmf]
    shutil.copy(fp_hpmf, out_dir_hpmf)
    shutil.copy(fp_labels, out_dir_labels)

CPU times: total: 0 ns
Wall time: 80.4 ms


In [40]:
len(os.listdir(out_dir_hpmf)) == n_test

True

In [41]:
len(os.listdir(out_dir_labels)) == n_test

True